# Keras学习
## keras 函数式API

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

函数式编程与keras.Sequential 相比而言，更加灵活
深度学习模型通常是一个DAG的层次模型，函数式API是其构建层次图的工具

In [2]:
# minst数据
# 输入数据 是784维的一维向量 ,批次大小通常需要忽略，因为需要根据实际的数据集大小来划分
inputs = keras.Input(shape=(784,))
# 也可以输入为一个32*32*3的tensor张量
# img_inputs = keras.Input(shape=(32,32,3))
inputs.dtype

tf.float32

In [3]:
inputs.shape

TensorShape([None, 784])

In [4]:
# 创建隐含层
dense = layers.Dense(64,activation='relu')
x = dense(inputs)
x = layers.Dense(64,activation='relu')(x)
outputs = layers.Dense(10)(x)

In [5]:
model = keras.Model(inputs=inputs,outputs=outputs,name='mnist_model')
model.summary()

Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 开始训练
# 
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000,784).astype("float32")/255
x_test = x_test.reshape(10000,784).astype("float32")/255



In [7]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),optimizer='adam',metrics=['accuracy'])

In [8]:
history=model.fit(x_train,y_train,batch_size=64,epochs=2,validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 2s 44us/sample - loss: 0.3533 - accuracy: 0.8986 - val_loss: 0.1916 - val_accuracy: 0.9436
Epoch 2/2
48000/48000 [==============================] - 1s 22us/sample - loss: 0.1593 - accuracy: 0.9529 - val_loss: 0.1380 - val_accuracy: 0.9602


In [9]:
test_scores = model.evaluate(x_test,y_test,verbose=2)
print('Test loss:%s' % test_scores[0])
print('Test accuracy:%s' % test_scores[1])

10000/1 - 0s - loss: 0.0932 - accuracy: 0.9618
Test loss:0.12943649917393923
Test accuracy:0.9618


In [10]:
# 保存模型和加载模型
model.save('mnist_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mnist_model\assets


In [11]:
del model
model = keras.models.load_model('mnist_model')
test_scores = model.evaluate(x_test,y_test,verbose=2)
print('Test loss:%s' % test_scores[0])
print('Test accuracy:%s' % test_scores[1])

10000/1 - 0s - loss: 0.0932 - accuracy: 0.9618
Test loss:0.12943649917393923
Test accuracy:0.9618


## 使用相同的图定义多个模型

In [12]:
encoder_input = keras.Input(shape=(28,28,1),name='img')
# Conv2D(filters,kernel_size) filters:是指的featuremap的层数，kernel_size指的是卷积核的大小比如 3*3，默认步长为（1，1），padding方式默认为valid
x=layers.Conv2D(16,3,activation='relu')(encoder_input)
x=layers.Conv2D(32,3,activation='relu')(x)
# 最大池化层 MaxPooling2D(pool_size,strides):pool_size 可以是tuple 或者是integer，代表池化层的大小，默认strides为None，即和pool_size相同，padding方式默认为valid
x=layers.MaxPooling2D(3)(x)
x=layers.Conv2D(32,3,activation='relu')(x)
x=layers.Conv2D(16,3,activation='relu')(x)
# GlobalMaxPooling2D() 全局池化，池化层的大小为整个feature-map的大小，最终得到一个1*1*feature-map个数的张量
encoder_output=layers.GlobalMaxPooling2D()(x)
encoder = keras.Model(encoder_input,encoder_output,name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                0   

In [13]:
x=layers.Reshape((4,4,1))(encoder_output)
# 反卷积 将数据扩充，卷积的输入输出，在这里可看作反卷积的 输出 输入
x=layers.Conv2DTranspose(16,3,activation='relu')(x)
x=layers.Conv2DTranspose(32,3,activation='relu')(x)
# 上采样
x=layers.UpSampling2D(3)(x)
x=layers.Conv2DTranspose(16,3,activation='relu')(x)
decoder_output=layers.Conv2DTranspose(1,3,activation='relu')(x)
autodecoder = keras.Model(encoder_input,decoder_output,name='autodecoder')
autodecoder.summary()

Model: "autodecoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                

定义好模型后 ，你可以像使用layer一样使用模型

## 构建复杂的图拓扑
### 定义有多个输入和输出的模型

In [14]:
num_tags=12
num_words=10000
num_departments=4
title_input=keras.Input(shape=(None,),name='title')
body_input=keras.Input(shape=(None,),name='body')
tags_input=keras.Input(shape=(num_tags,),name='tags')
# embedding
title_features=layers.Embedding(num_words,64)(title_input)
body_features=layers.Embedding(num_words,64)(body_input)
title_features=layers.LSTM(128)(title_features)
body_features=layers.LSTM(32)(body_features)
x=layers.concatenate([title_features,body_features,tags_input])
priority_pred=layers.Dense(1,name='priority')(x)
department_pred=layers.Dense(num_departments,name='department')(x)
model=keras.Model(inputs=[title_input,body_input,tags_input],outputs=[priority_pred,department_pred],name='multimodel')
model.summary()

Model: "multimodel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
body (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     640000      title[0][0]                      
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      body[0][0]                       
_________________________________________________________________________________________

In [15]:
keras.utils.plot_model(model,'multi_input_and_output_model.png',show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [16]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "priority": keras.losses.BinaryCrossentropy(from_logits=True),
        "department": keras.losses.CategoricalCrossentropy(from_logits=True),
    },
    loss_weights=[1.0, 0.2],
)

In [17]:
# Dummy input data
title_data = np.random.randint(num_words, size=(1280, 10))
body_data = np.random.randint(num_words, size=(1280, 100))
tags_data = np.random.randint(2, size=(1280, num_tags)).astype("float32")

# Dummy target data
priority_targets = np.random.random(size=(1280, 1))
dept_targets = np.random.randint(2, size=(1280, num_departments))

model.fit(
    {"title": title_data, "body": body_data, "tags": tags_data},
    {"priority": priority_targets, "department": dept_targets},
    epochs=2,
    batch_size=32,
)

Train on 1280 samples
Epoch 1/2
1280/1280 [==============================] - 4s 3ms/sample - loss: 1.3079 - priority_loss: 0.7019 - department_loss: 3.0302
Epoch 2/2
1280/1280 [==============================] - 1s 1ms/sample - loss: 1.2995 - priority_loss: 0.6997 - department_loss: 2.9992


## 构建一个简单的ResNet网络

In [18]:
inputs = keras.Input(shape=(32,32,3),name='img')
# 第一块block
x=layers.Conv2D(32,3,activation='relu')(inputs)
x=layers.Conv2D(64,3,activation='relu')(x)
block_1_output=layers.MaxPooling2D(3)(x)
# 第二块block
x=layers.Conv2D(64,3,activation='relu',padding='same')(block_1_output)
x=layers.Conv2D(64,3,activation='relu',padding='same')(x)
block_2_output=layers.add([x,block_1_output])
# 第三块block
x=layers.Conv2D(64,3,activation='relu',padding='same')(block_2_output)
x=layers.Conv2D(64,3,activation='relu',padding='same')(x)
block_3_output=layers.add([x,block_2_output])
# 输出
x=layers.Conv2D(64,3,activation='relu')(block_3_output)
x=layers.GlobalAveragePooling2D()(x)
x=layers.Dense(256,activation='relu')(x)
x=layers.Dropout(0.5)(x)
outputs=layers.Dense(10)(x)

model=keras.Model(inputs,outputs,name='cifar10_resnet')
model.summary()


Model: "cifar10_resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 30, 30, 32)   896         img[0][0]                        
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 28, 28, 64)   18496       conv2d_4[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 9, 9, 64)     0           conv2d_5[0][0]                   
_____________________________________________________________________________________

In [24]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float")/255.0
x_test = x_test.astype("float")/255.0
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [25]:
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),loss=keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['acc'])
model.fit(x_train[:1000],y_train[:1000],batch_size=64,epochs=100,validation_split=0.2,verbose=1)

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 3s 3ms/sample - loss: 2.0966 - acc: 0.2887 - val_loss: 1.7715 - val_acc: 0.3450
Epoch 2/100
800/800 [==============================] - 2s 2ms/sample - loss: 1.7425 - acc: 0.3363 - val_loss: 1.8373 - val_acc: 0.3400
Epoch 3/100
800/800 [==============================] - 2s 2ms/sample - loss: 1.6943 - acc: 0.3562 - val_loss: 1.7937 - val_acc: 0.3700
Epoch 4/100
800/800 [==============================] - 2s 2ms/sample - loss: 1.7168 - acc: 0.3613 - val_loss: 1.8567 - val_acc: 0.3300
Epoch 5/100
800/800 [==============================] - 2s 2ms/sample - loss: 1.6735 - acc: 0.3613 - val_loss: 1.6235 - val_acc: 0.3650
Epoch 6/100
800/800 [==============================] - 2s 2ms/sample - loss: 1.6991 - acc: 0.3700 - val_loss: 1.8516 - val_acc: 0.3250
Epoch 7/100
800/800 [==============================] - 2s 2ms/sample - loss: 1.5828 - acc: 0.4025 - val_loss: 1.6171 - val_acc: 0.3850
Epoch 8/1

Epoch 61/100
800/800 [==============================] - 2s 2ms/sample - loss: 0.7467 - acc: 0.7462 - val_loss: 1.8789 - val_acc: 0.4550
Epoch 62/100
800/800 [==============================] - 2s 2ms/sample - loss: 0.8130 - acc: 0.7287 - val_loss: 2.2586 - val_acc: 0.4150
Epoch 63/100
800/800 [==============================] - 2s 2ms/sample - loss: 0.5522 - acc: 0.8100 - val_loss: 2.0771 - val_acc: 0.4450
Epoch 64/100
800/800 [==============================] - 2s 2ms/sample - loss: 0.5910 - acc: 0.7837 - val_loss: 2.1771 - val_acc: 0.4300
Epoch 65/100
800/800 [==============================] - 2s 2ms/sample - loss: 0.8369 - acc: 0.7462 - val_loss: 2.1043 - val_acc: 0.4200
Epoch 66/100
800/800 [==============================] - 2s 2ms/sample - loss: 0.4968 - acc: 0.8288 - val_loss: 3.0058 - val_acc: 0.4050
Epoch 67/100
800/800 [==============================] - 2s 2ms/sample - loss: 0.7219 - acc: 0.7763 - val_loss: 2.2942 - val_acc: 0.4600
Epoch 68/100
800/800 [==========================

# 共享层

In [26]:
shared_embedding = keras.layers.Embedding(1000,100)
text_input_a = keras.layers.Input(shape=(None,),dtype="int32")
text_input_b = keras.layers.Input(shape=(None,),dtype="int32")
shared_a = shared_embedding(text_input_a)
shared_b = shared_embedding(text_input_b)
shared_a.shape

TensorShape([None, None, 100])

# 制定自己的layer

继承Layer类，并且实现call和build方法
call方法是最终layer输出的结果
build方法用来创建权重

In [30]:
class CustomDense(layers.Layer):
    def __init__(self,units=32):
        super(CustomDense,self).__init__()
        self.units = units
    def build(self,input_shape):
        self.w = self.add_weight(shape=(input_shape[-1],self.units),initializer='random_normal',trainable=True)
        self.b = self.add_weight(shape=(self.units,),initializer='random_normal',trainable=True)        
    def call(self,inputs):
        return tf.matmul(inputs,self.w) + self.b
inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)
model = keras.Model(inputs,outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
custom_dense_3 (CustomDense) (None, 10)                50        
Total params: 50
Trainable params: 50
Non-trainable params: 0
_________________________________________________________________
